In [1]:
import importlib
import os
import re
import sys
from collections import defaultdict, namedtuple
from dataclasses import dataclass, fields, asdict
from pathlib import Path
from mri_preproc.paths import hemond_data, init_paths
from mri_preproc import prepare_scans

Initialize paths (depends on what machine I'm running from)

In [2]:
init_paths.main()
from mri_preproc.paths.init_paths import DATA_HOME

Reload modules if I make any changes
- Double check that if I edit record.py, that reloading hd would reload those changes too, or if I need to use reload recursive

In [3]:
importlib.reload(hemond_data)
importlib.reload(prepare_scans)

<module 'mri_preproc.prepare_scans' from '/home/srs-9/Projects/ms_mri/mri_preproc/prepare_scans.py'>

In [ ]:
dataset = hemond_data.collect_choroid_dataset(DATA_HOME, suppress_output=True)

In [7]:
dataset[0].images

defaultdict(str,
            {'flair': PosixPath('/mnt/t/Data/3Tpioneer_bids/sub-ms1001/ses-20170215/flair.nii.gz'),
             'phase': PosixPath('/mnt/t/Data/3Tpioneer_bids/sub-ms1001/ses-20170215/phase.nii.gz'),
             't1': PosixPath('/mnt/t/Data/3Tpioneer_bids/sub-ms1001/ses-20170215/t1.nii.gz'),
             't1_gd': PosixPath('/mnt/t/Data/3Tpioneer_bids/sub-ms1001/ses-20170215/t1_gd.nii.gz')})

Convert the below into a function for processing a subject

In [9]:
subject = dataset[0]
proc_folder = subject.scan_folder() / "proc"
if not proc_folder.is_dir():
    os.makedirs(proc_folder)

/mnt/t/Data/3Tpioneer_bids/sub-1001/ses-20170215/proc


In [9]:
for i in range(10):
    nii_image = prepare_scans.load_scan(dataset[i].image)
    print(nii_image.shape)

(240, 300, 300)
(239, 300, 300)
(210, 300, 300)
(240, 300, 300)
(210, 300, 300)
(240, 300, 300)
(240, 300, 300)
(210, 300, 300)
(240, 300, 300)
(240, 300, 300)


In [9]:
for i in range(10):
    nii_image = prepare_scans.load_scan(dataset[i].image)
    print(nii_image.shape)

(240, 300, 300)
(239, 300, 300)
(210, 300, 300)
(240, 300, 300)
(210, 300, 300)
(240, 300, 300)
(240, 300, 300)
(210, 300, 300)
(240, 300, 300)
(240, 300, 300)


In [6]:
training_data = []
test_data = []

for scan in dataset:
    if scan.cond == 'tr':
        training_data.append({"image": scan.image, "label": scan.label})
    elif scan.cond == 'ts':
        test_data.append(scan.image)